# Octopod Ensemble Model Training Pipeline

As the seventh step of this tutorial, we will train an ensemble model using the two image models and one text model that we already trained.

This notebook was run on an AWS p3.2xlarge

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
import joblib
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from transformers import AdamW, BertTokenizer, get_cosine_schedule_with_warmup

from octopod.learner import MultiTaskLearner, MultiInputMultiTaskLearner
from octopod.dataloader import MultiDatasetLoader
from octopod.ensemble import OctopodEnsembleDataset, BertResnetEnsembleForMultiTaskClassification

## Load in train and validation datasets

First we load in the csv's we created in Step 1.
Remember to change the path if you stored your data somewhere other than the default.

In [4]:
TRAIN_GENDER_DF = pd.read_csv('/home/ec2-user/fashion_dataset/gender_train.csv')

In [5]:
VALID_GENDER_DF = pd.read_csv('/home/ec2-user/fashion_dataset/gender_valid.csv')

In [6]:
TRAIN_SEASON_DF = pd.read_csv('/home/ec2-user/fashion_dataset/season_train.csv')

In [7]:
VALID_SEASON_DF = pd.read_csv('/home/ec2-user/fashion_dataset/season_valid.csv')

You will most likely have to alter this to however big your batches can be on your machine

In [8]:
batch_size = 128

In [9]:
bert_tok = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

max_seq_length = 128 

In [10]:
gender_train_dataset = OctopodEnsembleDataset(
    text_inputs=TRAIN_GENDER_DF['productDisplayName'],
    img_inputs=TRAIN_GENDER_DF['image_urls'],
    y=TRAIN_GENDER_DF['gender_cat'],
    tokenizer=bert_tok,
    max_seq_length=max_seq_length,
    transform='train',
    crop_transform='train'

)
gender_valid_dataset = OctopodEnsembleDataset(
    text_inputs=VALID_GENDER_DF['productDisplayName'],
    img_inputs=VALID_GENDER_DF['image_urls'],
    y=VALID_GENDER_DF['gender_cat'],
    tokenizer=bert_tok,
    max_seq_length=max_seq_length,
    transform='val',
    crop_transform='val'

)

season_train_dataset = OctopodEnsembleDataset(
    text_inputs=TRAIN_SEASON_DF['productDisplayName'],
    img_inputs=TRAIN_SEASON_DF['image_urls'],
    y=TRAIN_SEASON_DF['season_cat'],
    tokenizer=bert_tok,
    max_seq_length=max_seq_length,
    transform='train',
    crop_transform='train'

)
season_valid_dataset = OctopodEnsembleDataset(
    text_inputs=VALID_SEASON_DF['productDisplayName'],
    img_inputs=VALID_SEASON_DF['image_urls'],
    y=VALID_SEASON_DF['season_cat'],
    tokenizer=bert_tok,
    max_seq_length=max_seq_length,
    transform='val',
    crop_transform='val'
)

We then put the datasets into a dictionary of dataloaders.

Each task is a key.

In [11]:
train_dataloaders_dict = {
    'gender': DataLoader(gender_train_dataset, batch_size=batch_size, shuffle=True, num_workers=6),
    'season': DataLoader(season_train_dataset, batch_size=batch_size, shuffle=True, num_workers=6),
}
valid_dataloaders_dict = {
    'gender': DataLoader(gender_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=6),
    'season': DataLoader(season_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=6),
}

In [12]:
TrainLoader = MultiDatasetLoader(loader_dict=train_dataloaders_dict)
len(TrainLoader)

366

In [13]:
ValidLoader = MultiDatasetLoader(
    loader_dict=valid_dataloaders_dict,
    shuffle=False
)
len(ValidLoader)

123

Create Model and Learner
===

Since the image model could potentially have multiple Resnets for different subsets of tasks, we need to create an `image_task_dict` that splits up the tasks grouped by the Resnet they use.

This version uses different resnets for gender and season.

In [14]:
image_task_dict = {
    'gender': {
        'gender': TRAIN_GENDER_DF['gender_cat'].nunique()    
    },
    'season': {
        'season': TRAIN_SEASON_DF['season_cat'].nunique()
    }  
}

We still need to create the `new_task_dict` for the learner.

In [15]:
new_task_dict = {
    'gender': TRAIN_GENDER_DF['gender_cat'].nunique(),
    'season': TRAIN_SEASON_DF['season_cat'].nunique()
}

In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


We first initialize the model by setting up the right shape with the image_task_dict.

In [17]:
model = BertResnetEnsembleForMultiTaskClassification(
    image_task_dict=image_task_dict
)

We then load in the existing models by specifying the folder where the models live and their id's.

In [18]:
resnet_model_id_dict = {
    'gender': 'GENDER_IMAGE_MODEL1',
    'season': 'SEASON_IMAGE_MODEL1'
}

In [20]:
model.load_core_models(
    folder='/home/ec2-user/fashion_dataset/models/',
    bert_model_id='TEXT_MODEL1',
    resnet_model_id_dict=resnet_model_id_dict
)

We've set some helper methods that will freeze the core bert and resnets for you if you only want to train the new layers. As with all other aspects of training, this is likely to require some experimentation to determine what works for your problem.

You will likely need to explore different values in this section to find some that work
for your particular model.

In [21]:
model.freeze_bert()
model.freeze_resnets()

lr_last = 1e-3
lr_main = 1e-5

lr_list = [
    {'params': model.bert.parameters(), 'lr': lr_main},
    {'params': model.dropout.parameters(), 'lr': lr_main},   
    {'params': model.image_resnets.parameters(), 'lr': lr_main},
    {'params': model.image_dense_layers.parameters(), 'lr': lr_main},
    {'params': model.ensemble_layers.parameters(), 'lr': lr_last},
    {'params': model.classifiers.parameters(), 'lr': lr_last},
]

optimizer = optim.Adam(lr_list)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size= 4, gamma= 0.1)

In [22]:
loss_function_dict = {'gender': 'categorical_cross_entropy', 'season': 'categorical_cross_entropy'}
metric_function_dict = {'gender': 'multi_class_acc', 'season': 'multi_class_acc'}

In [23]:
learn = MultiInputMultiTaskLearner(model, TrainLoader, ValidLoader, new_task_dict, loss_function_dict, metric_function_dict)

Train Model
===

As your model trains, you can see some output of how the model is performing overall and how it is doing on each individual task.

In [24]:
learn.fit(
    num_epochs=10,
    scheduler=exp_lr_scheduler,
    step_scheduler_on_batch=False,
    optimizer=optimizer,
    device=device,
    best_model=True
)

train_loss,val_loss,gender_train_loss,gender_val_loss,gender_multi_class_accuracy,season_train_loss,season_val_loss,season_multi_class_accuracy,time
0.306214,0.002668,0.047669,0.000235,0.990544,0.651101,0.005914,0.696951,06:22
0.280417,0.002853,0.030138,0.000222,0.992007,0.614277,0.006362,0.697702,06:23
0.277995,0.002576,0.028959,0.000265,0.990206,0.610196,0.005659,0.733143,06:23
0.272458,0.002689,0.025549,0.000273,0.989756,0.601823,0.005913,0.706713,06:23
0.264543,0.002518,0.023401,0.000195,0.992345,0.586214,0.005617,0.722181,06:23
0.261301,0.002597,0.020396,0.000205,0.992120,0.582656,0.005789,0.721730,06:23
0.261672,0.002496,0.021979,0.000200,0.991895,0.581410,0.005558,0.726686,06:23
0.262170,0.002442,0.019549,0.000212,0.991782,0.585815,0.005418,0.731641,06:23
0.259486,0.002600,0.020505,0.000204,0.991669,0.578274,0.005796,0.710617,06:24
0.259367,0.002497,0.020606,0.000211,0.991444,0.577864,0.005546,0.727887,06:23


Epoch 7 best model saved with loss of 0.0024424809962511063


You will need to check your specific use case to determine whether it is better to train all of your image tasks in one resnet or in multiple resnets. You won't necessarily need one resnet per task, particularly if some of your tasks are related.

Checking validation data
===

We provide a method on the learner called `get_val_preds`, which makes predictions on the validation data. You can then use this to analyze your model's performance in more detail.

In [25]:
pred_dict = learn.get_val_preds(device)

In [26]:
pred_dict

{'gender': {'y_true': array([4, 2, 2, ..., 2, 1, 2]),
  'y_pred': array([[9.3476007e-07, 2.2219403e-05, 4.8013021e-06, 1.1514738e-05,
          9.9996054e-01],
         [6.3747198e-06, 1.8497840e-06, 9.9998248e-01, 5.0397321e-06,
          4.2988258e-06],
         [9.0632973e-05, 8.3966652e-06, 9.9986410e-01, 1.6343623e-05,
          2.0512149e-05],
         ...,
         [2.1760125e-06, 5.8844546e-07, 9.9999499e-01, 9.2395891e-07,
          1.2700716e-06],
         [2.3211096e-04, 9.8485768e-01, 5.7467492e-05, 3.5564288e-05,
          1.4817212e-02],
         [1.6530554e-05, 2.5858185e-06, 9.9996901e-01, 8.9132454e-06,
          3.0193678e-06]], dtype=float32)},
 'season': {'y_true': array([0, 2, 2, ..., 1, 3, 2]),
  'y_pred': array([[0.56129366, 0.03476885, 0.3034806 , 0.10045694],
         [0.39057994, 0.0507347 , 0.5407597 , 0.01792566],
         [0.0328876 , 0.00295794, 0.9487496 , 0.01540487],
         ...,
         [0.1392722 , 0.03094479, 0.45717874, 0.37260425],
         [0.00

Save/Export Model
===

The ensemble model can also be saved or exported.

In [27]:
model.save(folder='/home/ec2-user/fashion_dataset/models/', model_id='ENSEMBLE_MODEL2')

In [28]:
model.export(folder='/home/ec2-user/fashion_dataset/models/', model_id='ENSEMBLE_MODEL2')